In [14]:
import itertools
import mdtraj as md
import mdtraj.testing



In [15]:
t = md.load('./data/pdbs/AF-C0H3V2-F1.pdb')
print(t)
hbonds = md.baker_hubbard(t)
for hbond in hbonds:
    print(hbond)

<mdtraj.Trajectory with 1 frames, 1105 atoms, 143 residues, without unitcells>


C:\Users\marik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mdtraj\formats\pdb\pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


In [16]:
label = lambda hbond: f"{t.topology.atom(hbond[0])} -- {t.topology.atom(hbond[2])}"

for hbond in hbonds:
    print(label(hbond))

In [473]:
def VdW_interaction(path, pqr_files=None, output = None):
    import numpy as np
    import os
    import scipy
    from scipy.spatial.distance import cdist
    Donor_list = [('GLN', 'NE2', 'HE21'), ('GLN', 'NE2', 'HE22'), ('GLU', 'OE2', 'HE2'), ('ASP', 'OD2', 'HD2'), ('ASN', 'ND2', 'HD21'), ('ASN', 'ND2', 'HD22'),
                    ('HIS', 'NE2', 'HE2'), ('HIS', 'ND1', 'HD2'), ('LYS', 'NZ', 'HZ1'), ('LYS', 'NZ', 'HZ2'), ('LYS', 'NZ', 'HZ3'), ('ARG', 'NE', 'HE'), 
                    ('ARG', 'NH1', 'HH11'), ('ARG', 'NH1', 'HH12'), ('ARG', 'NH2', 'HH21'), ('ARG', 'NH2', 'HH22'), ('SER', 'OG', 'HG'), ('THR', 'OG1', 'HG1'), 
                    ('TRP', 'NE1', 'HE1'), ('TYR', 'OH', 'HH')]
    Acceptor_list = [('GLN', 'OE1'), ('ASP', 'OD1'), ('ASP', 'OD2'), ('ASN', 'OD1')]
    
    if pqr_files is None:
        pqr_files = [f for f in os.listdir(path) if f.endswith('.pdb')]
    if isinstance(pqr_files, str):
        pqr_files = [pqr_files]
    for pqr_file in pqr_files:
        with open(os.path.join(path, 'C0H3Z2.pqr')) as f:
            HB_dic = {}
            Donor_array = np.empty((0, 4))
            H_array = np.empty((0, 4))
            Acceptor_array = np.empty((0, 4))
            aa_cache = []
            atom_cache = []
            test_count = 0
            for line in f:
                if line.startswith('ATOM'):
                    if not aa_cache:
                        aa_cache.append(line.split()[3])
                        aa_cache.append(line.split()[4])
                        atom_cache.append(line)
                    elif aa_cache[1] == line.split()[4]:
                        atom_cache.append(line)
                    elif aa_cache[1] != line.split()[4]:
                        test_count += 1             
                        for n in range(len(Donor_list)):
                            if aa_cache[0] == Donor_list[n][0]:
                                for i in range(len(atom_cache)):
                                    if Donor_list[n][1] == atom_cache[i].split()[2]:
                                        if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                            if any('HE2' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0) 
                                        elif ('ASP' == aa_cache[0]) and 'OD2' in atom_cache[i].split()[2]:
                                            if any('HD2' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0)                                             
                                        elif ('HIS' == aa_cache[0]) and 'ND1' in atom_cache[i].split()[2]:
                                            if any ('HD1' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0)
                                        elif ('HIS' == aa_cache[0]) and 'NE2' in atom_cache[i].split()[2]:
                                            if any('HE2' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0)
                                        else: 
                                            line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                            line_array = line_array.astype('float64')
                                            Donor_array = np.append(Donor_array, line_array, axis=0)         
                                    elif Donor_list[n][2] == atom_cache[i].split()[2]:
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        H_array = np.append(H_array, line_array, axis=0)
                        for n in range(len(Acceptor_list)):
                            if aa_cache[0] == Acceptor_list[n][0]:
                                for i in range(len(atom_cache)):
                                    if Acceptor_list[n][1] == atom_cache[i].split()[2]:
                                        if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                            if any('HE2' not in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                        elif ('ASP' == aa_cache[0]) and 'OD' in atom_cache[i].split()[2]:
                                            if any('HD' not in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                        else: 
                                            line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                            line_array = line_array.astype('float64')
                                            Acceptor_array = np.append(Acceptor_array, line_array, axis=0) 
            from helper_function import distance
            from helper_function import angle_calc
            angle = angle_calc(Donor_array, H_array, Acceptor_array)
            HB_dic[str(pqr_file).split('.')[1]] = angle
    return HB_dic
                                            



In [294]:
def distance (array1, array2, cutoff = None, remove_nan=True):
    from scipy.spatial.distance import cdist
    import numpy as np
    distance = cdist(array1[:,1:], array2[:,1:], metric='euclidean') #calculate distance
    distance = np.concatenate((np.array([array1[:,0]]).T, distance), axis=1) #add atom number from  array1
    distance = np.concatenate(((np.insert(np.array([array2[:,0]]), 0, None).reshape(-1,1)).T, distance), axis=0) #add atom number from array1
    if cutoff is not None:
        distance[1:, 1:][distance[1:, 1:] >= cutoff] = np.nan #set distance > cutoff to nan
        
    if remove_nan == False:
        return distance
    elif remove_nan == True:
        rows_with_nan = np.insert(np.array([np.all(np.isnan(distance[1:, 1:]), axis=1)]),0, None)
        rows_with_nan = np.insert(np.array([np.all(np.isnan(distance[1:, 1:]), axis=1)]),0, None) #find rows with all nan values
        cols_with_nan = np.insert(np.array([np.all(np.isnan(distance[1:, 1:]), axis=0)]),0, None) #find columns with all nan values
        distance = distance[~rows_with_nan, :] #delete rows with all nan values
        distance = distance[:, ~cols_with_nan] #delete columns with all nan values
        distance[:,0] = distance[:,0].astype('int')
        return distance
    else:
        raise ValueError('remove_nan must be either True or False')

In [471]:
def angle_calc(Donor_array, H_array, Acceptor_array):
    d_DH = np.full((0,2,2), fill_value = np.nan)
    for n in range(len(Donor_array)):
        DH_temp = distance(np.array([Donor_array[n,:]]), np.array([H_array[n,:]]))
        d_DH = np.concatenate((d_DH, DH_temp.reshape((1,) + DH_temp.shape)), axis=0)
    d_HA = distance(Donor_array, Acceptor_array, remove_nan=False, cutoff = 3.5)
    
    d_DA = distance(Donor_array, Acceptor_array, remove_nan=False)
    angle = np.full((d_DA.shape[0], d_DA.shape[1]), fill_value = np.nan)
    angle[0,:] = d_DA[0,:].T
    for n in range(d_DH.shape[0]):
            theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))
            theta[(theta < 100* np.pi/180) | (theta > np.pi)] = np.nan
            angle[n,0] = d_DA[n,0]
            angle[1:,1:][n,:] = theta 
    H_id = np.full((angle.shape[0], angle.shape[1]), fill_value = np.nan)
    H_id[1:,0] = d_DH[:,1,0]
    angle = np.dstack((angle, H_id))
    return angle


angle = angle_calc(Donor_array, H_array, Acceptor_array)
print(angle.shape)

(64, 12, 2)


C:\Users\marik\AppData\Local\Temp\ipykernel_1868\3944719170.py:18: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')
C:\Users\marik\AppData\Local\Temp\ipykernel_1868\3632117180.py:12: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))


In [467]:
print(angle[:,:,1])

[[  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [  28.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [  28.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [  28.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [  67.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 136.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 137.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 137.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 138.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 138.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 208.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 220.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 252.   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan]
 [ 253.   nan   nan   nan

In [170]:
d_DH = distance(Donor_array, H_array, cutoff =2.5 )
d_HA = distance(H_array, Acceptor_array)
d_DA = distance(Donor_array, Acceptor_array)

print(d_DH.shape)
print(d_HA.shape)
print(d_DA.shape)

(63, 63)
(64, 12)
(64, 12)


In [7]:
string = 'ATOM     12  HB3 MET     1     -20.987   0.463   6.535  0.0000 0.0000'
string2 ='ATOM      5  O   MET     1     -18.415  -1.135   7.336 -0.5500 1.4000'
list2 = [string.split()[4]]
print(list2)

if list2[0] == string2.split()[4]:
    print('yes')
else:
    print('no')


['1']
yes
